In [ ]:
"""This ipynb file convert semantic mask of to instance mask"""

In [ ]:
import numpy as np
from PIL import Image
import os
import scipy.io

In [ ]:
def read_mat(mat_file_name):
    import scipy.io
    import skimage.io as io
    key = 'ATmask'
    mat = scipy.io.loadmat(mat_file_name, mat_dtype=True, squeeze_me=True, struct_as_record=False)
    return mat[key];

In [ ]:
def convert_mat_annotations_to_png(mat_file_name):
    """ convert mat annotation file to png image 
    
    Parameters
    ----------
    masks_file_name
        Path to the matfile.
    
    """
    
    import scipy.io
    
    import skimage.io as io
    label_colours = [(128, 0, 0), (0, 128,0), (0, 0, 128), (128, 128, 0)]
    
    key = 'ATmask'
    
    mat = scipy.io.loadmat(mat_file_name, mat_dtype=True, squeeze_me=True, struct_as_record=False)
    h,w = mat[key].shape

    outputs = np.zeros((h,w,3), dtype = np.uint8)
    num_classes = 5

    img = Image.new('RGB', (h, w))
    pixels = img.load()
    for j_, j in enumerate(mat[key]):
        for k_, k in enumerate(j):
            if k < num_classes:
                pixels[k_,j_] = label_colours[int(k-1)]
    outputs = np.array(img)
    return outputs

In [ ]:
def convert_segmentation_array_to_png(mask, class_id):
    """ convert mat annotation file to png image 
    
    Parameters
    ----------
    masks_file_name
        Path to the matfile.
    
    """
    mask = np.transpose(mask)
    import scipy.io
    
    import skimage.io as io
    label_colours = [(128, 0, 0), (0, 128,0), (0, 0, 128), (128, 128, 0)]
    
    h,w = mask.shape

    outputs = np.zeros((h,w,3), dtype = np.uint8)
    num_classes = 4

    img = Image.new('RGB', (h, w))
    pixels = img.load()
    for j_, j in enumerate(mask):
        for k_, k in enumerate(j):
            if k == 1:
                pixels[j_,k_] = label_colours[int(class_id - 1)]
    outputs = np.array(img)
    return outputs

In [ ]:
def convert_array_to_png(mat):
    """ convert mat annotation file to png image 
    
    Parameters
    ----------
    masks_file_name
        Path to the matfile.
    
    """
    mat = np.transpose(mat)
    import scipy.io
    
    import skimage.io as io
    label_colours = [(128, 0, 0), (0, 128,0), (0, 0, 128), (128, 128, 0)]
    
    h,w = mat.shape

    outputs = np.zeros((h,w,3), dtype = np.uint8)
    num_classes = 4

    img = Image.new('RGB', (h, w))
    pixels = img.load()
    for j_, j in enumerate(mat):
        for k_, k in enumerate(j):
            if k <= num_classes:
                pixels[j_,k_] = label_colours[int(k-1)]
    outputs = np.array(img)
    return outputs

In [ ]:
class Unionfind:
    
    def __init__(self, n, mask):
        self.father = np.arange(n)
    
    def _find(self, x):
        if self.father[x] == x: 
            return x
        self.father[x] = self._find(self.father[x])
        return self.father[x]
    
    def connect(self, a, b):
        root_a = self._find(a)
        root_b = self._find(b)
        if (root_a != root_b):
            self.father[root_b] = root_a
            
    def set_father(self, pos, val):
        self.father[pos] = val

In [ ]:
data_path = '/data/wenyuan/Path_R_CNN/Data_Pre_Processing'
mat_file_name = os.path.join(data_path, 'cedars-224/masks/0000.mat')
matfile = read_mat(mat_file_name)

In [ ]:
outputs = convert_mat_annotations_to_png(mat_file_name)
im = Image.fromarray(outputs)
im.show()

In [ ]:
mask = matfile.astype(int)
r, c = mask.shape
my_union = Unionfind(r * c, mask.reshape(-1))

dr = [0, 1, -1, 0] # right, down, up, left
dc = [1, 0, 0, -1]
old = np.ones(r * c)
while(not np.array_equal(my_union.father,old)):   
    old = np.copy(my_union.father)
    for i in range(r):
        for j in range(c):
            if(mask[i, j] == 4):
                my_union.set_father(i * c + j, -1)
            else:
                for k in range(4):
                    if (i + dr[k] >= 0 and i + dr[k] < r and
                       j + dc[k] >= 0 and j + dc[k] < c and mask[i + dr[k], j + dc[k]] == mask[i, j]):
                        my_union.connect(i * c + j, (i + dr[k]) * c + j + dc[k])

In [ ]:
res = my_union.father.reshape((r, c))

In [ ]:
unique_set = np.copy(np.unique(res))
class_ids = []
hash_dict = {'4': 0, '1':1, '2':2, '3':3}
for index, val in enumerate(unique_set):
    if index == 0:
        stack = (res == val).astype(int)
    else:
        stack = np.dstack((stack, (res == val).astype(int)))
    pos = np.argwhere(res == val)[0]
    class_ids.append(hash_dict[str(mask[pos[0], pos[1]])])

In [ ]:
# save to mat file
res_dict = {'segmentation': stack, 'class_ids': class_ids}
scipy.io.savemat(os.path.join(data_path, 'cedars-224/masks/0000_instance.mat'), \
                 res_dict)